In [339]:
from requests import get, post, put
from dotenv import load_dotenv, find_dotenv
import os
import pandas as pd
load_dotenv(find_dotenv())


True

In [360]:
auth = f"Bearer BQDHY7A_g22I5lwW2RSmGucJHHZVffcnBe53j5D383jR7NXBcDIPEmfEsQ_Q9qbrPZPJPIcGFR6beFhG_F40kTV4oHoX59UnVfb6JtyPV0a3kH9Z3vuNohoisH5rAdUau6-BLk6SUL6quIJpa5NqNnpsprxV5tERSQk1uoH4YpMmAvL8eOnJg9O50Vftxh_6Ey9ePi682N2iVL-z9ChRqMvcCwE-Tm6ZZBN3xhc3ein818geYwbOkOWhXwZ7"
headers = {"Accept": "application/json",
           "Content-Type": "application/json", "Authorization": auth}


In [ ]:
artists = {}
songs = []


def get_total_song_count():
    playlist_res = get(
        f'https://api.spotify.com/v1/playlists/{os.getenv("PLAYLIST_ID")}', headers=headers)
    return playlist_res.json()["tracks"]["total"]


def add_genres(list, genres):
    for genre in genres:
        if genre not in list:
            list.append(genre)

    return list


def get_song_genres(song):
    genres = []
    for artist in song["track"]["artists"]:
        id = artist["id"]
        if id not in artists:
            artist_genres_res = get(
                f'https://api.spotify.com/v1/artists/{id}', headers=headers)
            artist_genres = artist_genres_res.json()["genres"]
            genres = add_genres(genres, artist_genres)
            artists[artist["name"]] = artist_genres
        else:
            genres = add_genres(genres, artists[id])

    return genres


def song_data(song):
    return song["track"]['id'], song["track"]['name'], song["track"]['popularity'], [artist["name"] for artist in song["track"]["artists"]]


In [ ]:


all_genres = []
for offset in range(0, get_total_song_count(), 100):
    all_genres_res = get(
        f'https://api.spotify.com/v1/playlists/{os.getenv("PLAYLIST_ID")}/tracks?limit=100&offset={offset}', headers=headers)
    for song in all_genres_res.json()["items"]:
        (id, name, popularity, artist), song_genres = song_data(
            song), get_song_genres(song)
        songs.append({"id": id, "name": name, "artists": artist,
                     "popularity": popularity, "genres": song_genres})
        all_genres = add_genres(all_genres, song_genres)


In [ ]:
playlist = pd.DataFrame(data=songs, columns=[
                        'id', 'name', 'artists', 'popularity', 'genres'])


In [ ]:
playlist


In [ ]:
all_artists = list(artists.keys())


In [ ]:
def genres_indexed(genres: list[str]) -> list[int]:
    indexed = []
    for all_genres_x in all_genres:

        continue_outer = False
        for genre in genres:
            index = all_genres_x == genre
            if index:
                continue_outer = True
                indexed.append(int(True))
                break

        if continue_outer:
            continue

        indexed.append(int(False))

    return indexed


def artists_indexed(artists: list[str]) -> list[int]:
    indexed = []
    for all_artists_x in all_artists:

        continue_outer = False
        for genre in artists:
            index = all_artists_x == genre
            if index:
                continue_outer = True
                indexed.append(int(True))
                break

        if continue_outer:
            continue

        indexed.append(int(False))

    return indexed


In [ ]:
playlist["genres_indexed"] = [genres_indexed(
    playlist["genres"][x]) for x in range(len(playlist["genres"]))]


In [ ]:
import numpy as np
playlist["genres_indexed"] = [np.array(
    playlist["genres_indexed"][x]) for x in range(len(playlist["genres_indexed"]))]


In [ ]:
playlist["artists_indexed"] = [np.array(artists_indexed(
    playlist["artists"][x])) for x in range(len(playlist["artists"]))]


In [ ]:
playlist


## KNN


In [ ]:
def knn_prepared_data(df):
    df = df[['id', 'name', 'genres', 'artists',
             'popularity', 'genres_indexed', 'artists_indexed']]
    list = []
    for index in range(len(df['id'])):
        list.append({'id': df['id'][index], 'name': df['name'][index], 'genres': df['genres'][index], 'artists': df['artists'][index],
                    'popularity': df['popularity'][index], 'genres_indexed': df['genres_indexed'][index], 'artists_indexed': df['artists_indexed'][index]})

    return list


In [332]:
from scipy import spatial


def ComputeDistance(a, b):
    genres_distance = spatial.distance.cosine(
        a['genres_indexed'], b['genres_indexed'])
    artists_distance = spatial.distance.cosine(
        a['artists_indexed'], b['artists_indexed'])
    popularity_distance = abs(a['popularity'] - b['popularity'])
    # print(genres_distance, artists_distance, popularity_distance)
    return genres_distance + artists_distance * 0.3 + 0.0065 * popularity_distance


In [333]:
song_dict = knn_prepared_data(playlist)


In [334]:
import operator


def getNeighbors(song, K):
    distances = []
    for song_index, song_value in enumerate(song_dict):
        if (song_index != song):
            dist = ComputeDistance(song_dict[song], song_value)
            distances.append((song_index, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors


## Song options

### Choose one of these to get the 50 closest songs to it


In [335]:
playlist['name']


0      Die For You (feat. Dominic Fike)
1         Get You The Moon (feat. Snøw)
2                           Out Of Love
3          I'm Not Enough and I'm Sorry
4                                change
                     ...               
980      i love you but you make me cry
981              thenineteenseventyfive
982    I'm Leaving, Sorry For Your Loss
983               Something Super Sweet
984                  Cut My Fingers Off
Name: name, Length: 985, dtype: object

In [365]:
# Set the song name below
top_5 = get('https://api.spotify.com/v1/me/top/tracks?time_range=short_term&limit=5', headers=headers).json()
song_name = 'All Without You'
top_5_names = map(lambda song: song['name'],top_5['items'])
for i in top_5_names:
    if i in playlist['name']:
        song_name = i
        break

In [338]:

K = 50
item = playlist[[playlist['name'][x] ==
                 song_name for x in range(len(playlist['name']))]]
index = item.index[0]
neighbors = getNeighbors(index, K)
print(song_dict[index]['name'], song_dict[index]['artists'],
      song_dict[index]['genres'], song_dict[index]['popularity'])
for neighbor, index in zip(neighbors, range(1, K + 1)):
    print(
        f'{index}. {song_dict[neighbor]["name"]} \t {song_dict[neighbor]["artists"]} \t {song_dict[neighbor]["genres"]} \t {song_dict[neighbor]["popularity"]}')


All Without You ['Laeland', 'Snøw'] ['lo-fi chill', 'sad lo-fi', 'sad rap'] 48
1. I'm inlove but... 	 ['Laeland', 'Snøw'] 	 ['lo-fi chill', 'sad lo-fi', 'sad rap'] 	 60
2. Tell Me You Want Me 	 ['Snøw', 'Nuxe'] 	 ['sad lo-fi', 'sad rap', 'lo-fi chill'] 	 38
3. this is bs. 	 ['Skinny Atlas', 'Laeland', 'Snøw'] 	 ['emo rap', 'lo-fi chill', 'sad lo-fi', 'sad rap'] 	 53
4. I never wanna lose you 	 ['Teqkoi', 'Snøw'] 	 ['lo-fi chill', 'sad lo-fi', 'sad rap'] 	 60
5. Ryu Kick 	 ['Snøw', 'Teqkoi'] 	 ['sad lo-fi', 'sad rap', 'lo-fi chill'] 	 32
6. If I Die 	 ['Snøw', 'Josiah MacCartney', 'Criggoh', 'Julia Alexa'] 	 ['sad lo-fi', 'sad rap', 'lo-fi chill'] 	 38
7. Life's a Blur 	 ['Teqkoi', 'Snøw'] 	 ['lo-fi chill', 'sad lo-fi', 'sad rap'] 	 31
8. I'm Not Enough and I'm Sorry 	 ['Teqkoi', 'Snøw'] 	 ['lo-fi chill', 'sad lo-fi', 'sad rap'] 	 69
9. summer nights 	 ['Skinny Atlas', 'Laeland', 'Snøw'] 	 ['emo rap', 'lo-fi chill', 'sad lo-fi', 'sad rap'] 	 63
10. You Don't Want Me Anymore 	 ['Snøw', '

C:\Python39\lib\site-packages\scipy\spatial\distance.py:699: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [358]:
import json
data = {"name": f"\'{song_name}\' Related", "description": f"Songs related to {song_name}", "public": True}
playlist_creation = post(f'https://api.spotify.com/v1/users/{os.getenv("USER_ID")}/playlists', headers=headers, data=json.dumps(data))
new_id = playlist_creation.json()['id']

In [359]:
item = playlist[[playlist['name'][x] ==
                 song_name for x in range(len(playlist['name']))]]
index = item.index[0]
song_uris = f'spotify:track:{song_dict[index]["id"]}'
for neighbor in neighbors:
    song_uris += f',spotify:track:{song_dict[neighbor]["id"]}'

add_songs_req = post(f'https://api.spotify.com/v1/playlists/{new_id}/tracks?uris={song_uris}', headers=headers, data=json.dumps({}))
add_songs_req.json()

{'snapshot_id': 'MyxlMGRmYjA5YTg5YjdhMzA1OTdkNzVjMDAzNDk2NDIzMjI2NTdjYmU4'}